In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

import os
# set cuda params
# 'TORCH_HOME'directory will be used to save origenal esm-1v weights
os.environ['TORCH_HOME'] = "./torch_hub"
#os.environ['CUDA_VISIBLE_DEVICES'] = "4"

In [3]:
# load pre-trained model
import esm

# database UniRef90
model_esm1v, esm1v_alphabet = esm.pretrained.esm1v_t33_650M_UR90S_1()

batch_converter = esm1v_alphabet.get_batch_converter()

/home/yuan/anaconda3/envs/py311/lib/python3.11/site-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


In [4]:

from custom_dataset import PdbDataset

agg_names = {'resi_pos': list, 'resi_aa': list, 'contact_number': list}

# train data
train_set = pd.read_csv('./data/sema_2.0/train_set.csv')
agg_names = {'resi_pos': list, 'resi_aa': list, 'contact_number': list}
train_set = train_set.groupby('pdb_id_chain').agg(agg_names).reset_index()
train_ds = PdbDataset(batch_converter, train_set[['resi_aa', 'contact_number']])

# test data
test_set = pd.read_csv('./data/sema_2.0/test_set.csv')
agg_names = {'resi_pos': list, 'resi_aa': list, 'contact_number_binary': list}
test_set = test_set.groupby('pdb_id_chain').agg(agg_names).reset_index()
test_ds = PdbDataset(batch_converter, test_set[['resi_aa', 'contact_number_binary']])

print(len(train_ds), len(test_ds))

1544 101


In [5]:
train_set.head()

,pdb_id_chain,resi_pos,resi_aa,contact_number
0,1A3R_P,"[156.0, 157.0, 158.0, 159.0, 160.0, 161.0, 162...","[V, K, A, E, T, R, L, N, P, D, L, Q, P, T, E, X]","[2.70805020110221, 2.833213344056216, 2.197224..."
1,1ACY_P,"[nan, nan, nan, nan, nan, nan, nan, 315.0, 316...","[Y, N, K, R, K, R, I, H, I, G, P, G, R, A, F, ...","[-100.0, -100.0, -100.0, -100.0, -100.0, -100...."
2,1AFV_B,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","[P, I, V, Q, N, L, Q, G, Q, M, V, H, Q, A, I, ...","[-100.0, -100.0, -100.0, -100.0, -100.0, -100...."
3,1AHW_F,"[nan, nan, nan, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","[S, G, T, T, N, T, V, A, A, Y, N, L, T, W, K, ...","[-100.0, -100.0, -100.0, -100.0, -100.0, -100...."
4,1AI1_P,"[nan, nan, nan, nan, nan, nan, nan, 315.0, 316...","[Y, N, K, R, K, R, I, H, I, G, P, G, R, A, F, ...","[-100.0, -100.0, -100.0, -100.0, -100.0, -100...."


In [6]:
train_ds[0]

{'token_ids': tensor([[ 0,  7, 15,  5,  9, 11, 10,  4, 17, 14, 13,  4, 16, 14, 11,  9, 24,  2]]),
 'labels': tensor([[   2.7090,    2.8340,    2.1973,    3.3320,    2.8340,    3.6113,
             3.0918,    2.9961,    2.1973,    2.3027,    2.6387,    2.3027,
             2.1973,    2.6387,    2.0801, -100.0000]], dtype=torch.float64)}